## Step 0 import necessary module and load data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

import time
import datetime
import os

#import datacleaningp1 as dcp
import dcpv2
reload(dcpv2)

In [ ]:
# load training data
data_train_all_v1 = pd.read_csv('TrainingData_fixed_4_test_1.csv')
print 'Training data read sucessfully'
display(data_train_all_v1.describe())

In [ ]:
# load test data
data_test_all_v1 = pd.read_csv('test_data_1.csv')
print 'Test data read successfully'
display(data_test_all_v1.describe())

In [ ]:
data_test_backup = data_test_all_v1.copy()
data_test_all_v1.drop(['date'],axis = 1,inplace = True)

## Step 1 feature selection and linear regression

In [ ]:
### shuffle training data
from sklearn.utils import shuffle
data_train_all = data_train_all_v1.copy()
data_train_all = shuffle(data_train_all,random_state = 42)

### split label from features
label = data_train_all['gaps']
data_train_all.drop(['gaps'], axis = 1, inplace= True)

### drop apparently irrelavant features
data_train_all.drop(['Weather_8','Weather_6'],axis= 1,inplace= True)
district_label = data_train_all['district_id']
#data_train_all.drop(['district_id'],axis= 1,inplace= True)

### feature_preprocessing
feature_cols = data_train_all.columns
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
data_train_scaled = min_max_scaler.fit_transform(data_train_all)
data_train_scaled = pd.DataFrame(data = data_train_scaled,columns = feature_cols)
#data_train_scaled['district_id'] = district_label
feature_cols = data_train_scaled.columns
print feature_cols

### train_test_split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train_scaled, label, test_size=0.3, random_state=42)

In [ ]:
### feature selection using Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV,Lasso
from sklearn.metrics import mean_absolute_error,r2_score

reg_lasso = LassoCV(max_iter=5000)
sfm = SelectFromModel(reg_lasso, threshold='0.01*mean') ### feature weights lower than threshhold will be removed
sfm.fit(X_train, y_train)

mask = sfm.get_support()
feature_selected = feature_cols[mask]
print feature_selected

X_train2 = X_train[feature_selected]
X_test2 = X_test[feature_selected]

### Train a ridge regression model
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV

rrm = linear_model.Ridge()
#parameters = {'alpha':np.logspace(0,2,10)}
parameters = {'alpha':np.linspace(0,10,10)}
grid_rr = GridSearchCV(rrm, parameters,scoring = 'mean_absolute_error',cv = 10)

grid_rr.fit(X_train2,y_train)

rr_final = grid_rr.best_estimator_
print rr_final.get_params()

y_pred = rr_final.predict(X_train2)
print 'r-square on training set: ',r2_score(y_train,y_pred)
print 'mae on training set: ',mean_absolute_error(y_train,y_pred)
y_pred = rr_final.predict(X_test2)
print 'r-square on test set: ',r2_score(y_test,y_pred)
print 'mae on test set: ' ,mean_absolute_error(y_test,y_pred)

In [ ]:
### measure performance on real test set
data_test_all = data_test_all_v1.copy()
district_label_test = data_test_all['district_id']
#data_test_all.drop(['district_id','Weather_6','Weather_8'],axis = 1,inplace = True)
data_test_all.drop(['Weather_6','Weather_8'],axis = 1,inplace = True)
feature_cols = data_test_all.columns
data_test_scaled = min_max_scaler.transform(data_test_all)
data_test_scaled = pd.DataFrame(data = data_test_scaled,columns = feature_cols)
#data_test_scaled['district_id'] = district_label_test

data_test2 = data_test_scaled[feature_selected]
gaps_pred = rr_final.predict(data_test2)

data_test_backup['gaps'] = gaps_pred

import score
reload(score)
output_test_pred = score.output_required_format(data_test_backup)
output_test_pred['district_id'] = output_test_pred['district_id'].apply(dcpv2.num2str)
output_test_pred.to_csv('test_pred_1.csv',index = False)
score.measure_performance_mae()
score.measure_performance_mape()

## Step 2 Train better kNN with selected features

In [ ]:
data_train_knn = data_train_scaled[feature_selected]
data_test_knn = data_test_scaled[feature_selected]
# label is still label

#data_train_knn['district_id'] = district_label
#data_test_knn['district_id'] = district_label_test
#data_train_knn['time'] = data_train_all_v1['time']
#data_test_knn['time'] = data_test_backup['time']
display(data_train_knn.head())
display(data_test_knn.head())

In [ ]:
district_number = 66
#test_set_size = 0.3 #use all training data
num_neighbors = 25

#data_train_all = data_train_all_v2

### import necessary moduels in sklearn
#from sklearn.utils import shuffle
#from sklearn import preprocessing
#from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error,r2_score
import dcpv2
reload(dcpv2)
import score
reload(score)
###

neigh = KNeighborsRegressor(n_neighbors= num_neighbors)
neigh.fit(data_train_knn,label)

gaps_pred = neigh.predict(data_test_knn)

data_test_backup['gaps'] = gaps_pred
output_test_pred = score.output_required_format(data_test_backup)
output_test_pred['district_id'] = output_test_pred['district_id'].apply(dcpv2.num2str)
#display(output_test_pred)
output_test_pred.to_csv('test_pred_2.csv',index = False,header = False)
score.measure_performance_mae(result_file = 'test_pred_2.csv')
score.measure_performance_mape(result_file = 'test_pred_2.csv')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
# Fit regression model
#params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 1,
          #'learning_rate': 0.01, 'loss': 'ls'}
#gbr = GradientBoostingRegressor(**params)
gbr = GradientBoostingRegressor(loss = 'lad')
params_grid = {'n_estimators': [10,30,100,300], 'max_depth': np.arange(10)+2,
          'learning_rate': [0.05,0.1,0.15,0.45]}

gbr_grid =  GridSearchCV(gbr, params_grid,scoring = 'mean_absolute_error',cv = 10)

gbr_grid.fit(data_train_knn,label)

gbr_final = gbr_grid.best_estimator_
print gbr_final.get_params()

gaps_pred = gbr_final.predict(data_test_knn)
gbr.fit(data_train_knn,label)

#gaps_pred = gbr.predict(data_test_knn)
data_test_backup['gaps'] = gaps_pred
output_test_pred = score.output_required_format(data_test_backup)
output_test_pred['district_id'] = output_test_pred['district_id'].apply(dcpv2.num2str)
#display(output_test_pred)
output_test_pred.to_csv('test_pred_3.csv',index = False,header = False)
score.measure_performance_mae(result_file = 'test_pred_3.csv')
score.measure_performance_mape(result_file = 'test_pred_3.csv')

In [ ]:
y_train = gbr_final.predict(data_train_knn)
print mean_absolute_error(label,y_train)